In [ ]:
# Training model
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epoch_loss_record = []
    epoch_acc_record = []
    poison_identified_record = []
    epoch_recall_record = []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0.0
            poison_identified = 0.0
            poison_sum = 0.0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                poison_identified += torch.sum((preds == labels.data) * (labels.data == 1))
                poison_sum += torch.sum(labels.data == 1)
            if phase == 'train':
                scheduler.step()

            # TODO: save the epoch loss, make figs later
            # when epoch loss stops decresing, it's time to change the learning rate scheduler

            epoch_loss = running_loss / dataset_sizes[phase]
            # epoch_acc = running_corrects.double() / dataset_sizes[phase]  # For Nvidia GPU
            epoch_acc = running_corrects.float() / dataset_sizes[phase]  # For mac m1 chip
            epoch_recall = poison_identified / poison_sum

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} Recall: {epoch_recall:.4f}')
            epoch_loss_record.append(epoch_loss)
            epoch_acc_record.append(epoch_acc)
            poison_identified_record.append(poison_identified)
            epoch_recall_record.append(epoch_recall)

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_loss_record, epoch_acc_record, poison_identified_record, epoch_recall_record